In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
from glob import glob
import upsilon
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import scipy.interpolate as interpolate
from scipy.optimize import curve_fit, least_squares
from statistics import mean, median
import pandas as pd
from dl import queryClient as qc
import collections
from astropy import stats
from astropy.table import Table
import os
import utils

-------------------------------
* WARNING: No pyFFTW detected *
-------------------------------


In [3]:
if not os.path.exists('results2/plots'):
    os.makedirs('results2/plots')

In [4]:
def get_data(objname):
    df=qc.query(sql="""SELECT meas.* 
                     FROM nsc_dr2.meas
                     WHERE objectid='{:s}'""".format(objname),
              fmt='pandas',
              profile='db01')
    order = ['u','g','r','i','z']
    best_periods = []
    crv=[]
    fltrs=[]
    for f in order:
        selfltr = (df['filter'] == f)
        selfwhm = (df['fwhm'] <= 4.0)
        sel = selfltr & selfwhm
        t = df['mjd'][sel].values
        y = df['mag_auto'][sel].values
        dy = df['magerr_auto'][sel].values
        if len(t) < 25:
            continue

        best_periods.append(get_ls_period(t,y,objname=objname,f=f))
        crvi = np.vstack((t,y,dy)).T
        crv.append(crvi[np.argsort(crvi[:,0])])
        fltrs.append(f)
    period = 0
    for p in best_periods:
        period += p/len(best_periods)
    return crv, period, fltrs
        
def get_ls_period(t,y,min_freq=1./1.,max_freq=1./0.1,objname='_',f=''):
    """Use Lomb-Scargle periodogram to get an estimate on period"""
    
    ls = stats.LombScargle(t, y)
    frequency, power = ls.autopower(minimum_frequency=min_freq,maximum_frequency=max_freq)
    period = 1./frequency # period is the inverse of frequency
    
    best_period = period[np.argmax(power)]
    
#     plot_periodogram(period,power,best_period,objname=objname,f=f)
    return best_period

In [5]:
gldorig = np.loadtxt('goldsample\golden_original.txt',delimiter=',',dtype=str)
gldrrab = np.loadtxt('goldsample\golden_RRab.txt',delimiter=',',dtype=str)
gldall  = np.union1d(gldorig,gldrrab)
gldelse = list(set(gldall) - set(gldrrab))

In [1]:
# t = Table.read('p_amp.fits')
# # t['period'] = -99.99
# t['type']   = '          '
# t['utyp']   = '          '
# t['uprob']  = -99.99
# t['uflag']  = -1
# t['undat']  = 0
# t['gtyp']   = '          '
# t['gprob']  = -99.99
# t['gflag']  = -1
# t['gndat']  = 0
# t['rtyp']   = '          '
# t['rprob']  = -99.99
# t['rflag']  = -1
# t['rndat']  = 0
# t['ityp']   = '          '
# t['iprob']  = -99.99
# t['iflag']  = -1
# t['indat']  = 0
# t['ztyp']   = '          '
# t['zprob']  = -99.99
# t['zflag']  = -1
# t['zndat']  = 0

In [ ]:
# t = Table.read("rrl_res_raw.fits",format='fits')
# newones = set(gldall)-set(gldrrab)
# for n,name in enumerate(gldall):
#     crv,p_est,fltrs = get_data(name)
#     periods = []
#     for i,f in enumerate(fltrs):
    
#         rf_model = upsilon.load_rf_model()
#         efeat = upsilon.ExtractFeatures(crv[i][:,0],crv[i][:,1],crv[i][:,2])
#         efeat.run()
#         feats = efeat.get_features()
#         rf_model = upsilon.load_rf_model()
#         label, probability, flag = upsilon.predict(rf_model, feats)
                
#         x = fdict[fltrs[i]]
#         t[n]['{}typ'.format(f)] = label
#         t[n]['{}prob'.format(f)] = probability
#         t[n]['{}flag'.format(f)] = flag
#         t[n]['{}ndat'.format(f)] = len(crv[i])
#         periods.append(feats['period'])

#     t[n]['period'] = np.mean(periods)
#     uprob = t[n]['uprob']
#     utyp  = t[n]['utyp']
#     gprob = t[n]['gprob']
#     gtyp  = t[n]['gtyp']
#     rprob = t[n]['rprob']
#     rtyp  = t[n]['rtyp']
#     iprob = t[n]['iprob']
#     ityp  = t[n]['ityp']
#     zprob = t[n]['zprob']
#     ztyp  = t[n]['ztyp']
    
#     m = np.argmax(np.array([[uprob,gprob,rprob,iprob,zprob]]))
#     maxtyp = [utyp,gtyp,rtyp,ityp,ztyp][m]
    
#     t[n]['type'] = maxtyp

# import os

# if os.path.exists("rrl_res.fits"):
#     os.remove("rrl_res.fits")
# t.write("rrl_res.fits",format='fits')

In [ ]:
t

In [ ]:
collections.Counter(t['type'])

In [ ]:
# import csv
# parampaths = glob('results2/*param.csv')
# nms     = []
# params  = []
# pars    = []
# periods = []
# for path in parampaths:
#     param = []
#     fpars = []
#     with open(path, newline='') as csvfile:
#         csvreader = csv.reader(csvfile, delimiter=',')
#         b     = True
#         first = True
#         for row in csvreader:
#             if row[0] == '---':break
#             if first:
#                 periods.append(float(row[2][2:]))
#                 first = False
#                 continue
#             if b:
#                 param.append(row)
#             else:
#                 fpars.append(row)
#             b = not b
#     param = np.array(param)
#     fpars = np.array(fpars)
#     nms.append(path[9:-10])
#     params.append(param)
#     pars.append(fpars)
# nms = np.array(nms)
# pdict = {nms[i]:periods[i] for i in range(len(nms))}

In [ ]:
amps  = []
namps = []
for i in range(len(pars)):
    famps = []
    for j in range(len(pars[i])):
        famps.append(float(pars[i][j,1][4:]))
    amps.append(np.array(famps))
    
    namps.append(amps[-1]/np.mean(amps[-1]))

In [ ]:
def

In [ ]:
figall, axall = plt.subplots()
figu, axu = plt.subplots()
figg, axg = plt.subplots()
figr, axr = plt.subplots()
figi, axi = plt.subplots()
figz, axz = plt.subplots()
for i in range(len(ceph_names)):
    p = pdict[ceph_names[i]]
    for fltr, crv in crvd[ceph_names[i]].items():
        axall.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]),s=1)
        if fltr == 'u':
            axu.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]),s=1)
        if fltr == 'g':
            axg.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]),s=1)
        if fltr == 'r':
            axr.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]),s=1)
        if fltr == 'i':
            axi.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]),s=1)
        if fltr == 'z':
            axz.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]),s=1)

In [ ]:
coeffs = np.polyfit(crv[:,0],crv[:,1],1)
coeffs

In [ ]:
x = np.arange(len(coeffs))
y = np.array([np.sum(np.array([coeffs[i]*(j**i) for i in range(len(coeffs))])) for j in x])

In [ ]:
ceph_inds = []
for i,name in enumerate(t['id']):
    if name in ceph_names:
        ceph_inds.append(i)

In [ ]:
t[ceph_inds]

In [ ]:
for i in range(len(ceph_names)):
    if ceph_names[i] in ['189344_10414','190269_170897','191137_70691','191515_128100','151560_15723','192287_129783']:
        print(i)

In [ ]:
phshft = np.zeros(36)
phshft[0]  += .02
phshft[11] += .025
phshft[23] += .025
phshft[25] += .035
phshft[28] += .02
phshft[32] -= .02

amp = np.ones(36)
amp[0]  = 1.05
amp[9]  = 0.95
amp[19] = 1.08
amp[20] = 1.05
amp[12] = 1.25
amp[22] = 1.7

yoffset = np.zeros(36)
yoffset[14] = -.175

In [ ]:
fig, ax = plt.subplots(36,figsize=(15,150))
count = 0
tt = []
yy = []
plist = []
for i in range(36):
    p = pdict[ceph_names[i]]
    plist.append(p)
    
    if 'g' in crvd[ceph_names[i]].keys():
        crv = crvd[ceph_names[i]]['g']
        tt.append((crv[:,0]/p+phshft[i])%1)
        yy.append(crv[:,1]/max(crv[:,1]))
        count = count + 1
        ax[i].scatter((crv[:,0]/p+phshft[i])%1,amp[i]*crv[:,1]/max(crv[:,1]),label=i,s=20)
        ax[i].set_ylim(-.1,1.1)
    
    
    for j in [25,26,28,29,32]:
        pj = pdict[ceph_names[j]]
        crvj = crvd[ceph_names[j]]['g']
        ax[i].scatter((crvj[:,0]/pj+phshft[j])%1,amp[j]*crvj[:,1]/max(crvj[:,1]),label=j,s=2)

    ax[i].legend()

[35] no g<br>
[34,15,26,28,31],[9?,18?,23?,25?,29?]<br>
[33,9,10,11,19,27,28],[2?,4?,23?,29?]<br>
[32,29],[13?,17?,18?,28?,30?],[~5,~6,~7]<br>
[31,9,,10,26,34],[28?]<br>
[30,15,25,26,32],[2?,7?,17?,18?]<br>
[29,0,15,25,26,28],[6?,7?,17?,18?,32?,34?,30?]<br>
[28,19,26,29,31],[7?,15?,25?,34?]<br>
[27,19],[1?,4?,11?]<br>
[26,18,28,29,32],[13?,17?,25?]
[25,15],[0?,14?,17?,18?,26?,28?,29?]<br>
[24] no g<br>
[23] Left side is off<br>
[22,5,20],[1?,6?,7?,13?]<br>
[21] Bad<br>
[20,5,22],[13?]<br>
[19,2,9,10],[11?,33?]<br>
[18,9,13,17,26,29],[6?,7?,15?,22,25?,28?]<br>
[17,13,18,22,26,32],[5?,6?,20?,25?,28?,29?,30?]<br>
[16,4,11,12],[27?]<br>
[15,0,7,25,26,28,29,30,32],[14?,18?,31?,34?]<br>
[14,0],[25?,28?,29?]<br>
[13,1,5,6,17,18,20,22,25,26,29,32],[7?,28?,30?,31?]<br>
[12,2,16,9,19,33],[4?,8?,10?,11?]<br>
[11,9,10,19,33],[2?,4?,16?,23?,27?]<br>
[10,2,9,11,19,22,31],[13?]<br>
[9,2,11,19,22,23,26,28,29,33,34]<br>
[8] Bad<br>
[7,0,6,15,22,29]<br>
[6,0,7,13,28],[20?,22?]<br>
[5,1,6,13,20,27],[22?]<br>
[4,9,10,11,12,16,19,27,33]<br>
[3]<br>
[2,4,9,10,11,19],[23?]<br>
[1,5,20,27]<br>
[0,6,7,14,25,28,29,32]<br>
RRLab?   t [17,18]<br>
irr [3,21,8]<br>

[17,13,18,22,26,32]<br>
13: good on right side, bad in middle, maybe RRL ab?<br>
17: Looks like RRL ab<br>
22: Messy, hard to tell amplitude due to outlier

In [ ]:
fig, ax = plt.subplots(2,figsize=(10,7))
for i in [25,26,28,29,32]:
    if 'g' in crvd[ceph_names[i]].keys():
        crv = crvd[ceph_names[i]]['g']
        p = pdict[ceph_names[i]]
        tt.append((crv[:,0]/p+phshft[i])%1)
        yy.append(crv[:,1]/max(crv[:,1]))
        size=5
        ax[0].scatter((crv[:,0]/p+phshft[i])%1,amp[i]*crv[:,1]/max(crv[:,1])+yoffset[i],label=i,s=size)
        ax[1].scatter((crv[:,0]/p+phshft[i])%1,amp[i]*crv[:,1]/max(crv[:,1])+yoffset[i],label=i,s=2,c='black')

for j in []:
    if 'g' in crvd[ceph_names[j]].keys():
        crv = crvd[ceph_names[j]]['g']
        p = pdict[ceph_names[j]]
        tt.append((crv[:,0]/p+phshft[j])%1)
        yy.append(crv[:,1]/max(crv[:,1]))
        ax[1].scatter((crv[:,0]/p+phshft[j])%1,amp[j]*crv[:,1]/max(crv[:,1])+yoffset[j],label=j,s=30)

ax[0].set_ylim(-.1,1.1)
ax[0].legend()
ax[1].set_ylim(-.1,1.1)
ax[1].legend()

In [ ]:
fig, ax = plt.subplots(2,figsize=(10,7))
for i in [2,9,10,19]:#[2,9,10,11,19]:
    if 'g' in crvd[ceph_names[i]].keys():
        crv = crvd[ceph_names[i]]['g']
        p = pdict[ceph_names[i]]
        tt.append((crv[:,0]/p+phshft[i])%1)
        yy.append(crv[:,1]/max(crv[:,1]))
        size=5
        ax[0].scatter((crv[:,0]/p+phshft[i])%1,amp[i]*crv[:,1]/max(crv[:,1])+yoffset[i],label=i,s=size)
        ax[1].scatter((crv[:,0]/p+phshft[i])%1,amp[i]*crv[:,1]/max(crv[:,1])+yoffset[i],label=i,s=2,c='black')

for j in [11]:
    if 'g' in crvd[ceph_names[j]].keys():
        crv = crvd[ceph_names[j]]['g']
        p = pdict[ceph_names[j]]
        tt.append((crv[:,0]/p+phshft[j])%1)
        yy.append(crv[:,1]/max(crv[:,1]))
        ax[1].scatter((crv[:,0]/p+phshft[j])%1,amp[j]*crv[:,1]/max(crv[:,1])+yoffset[j],label=j,s=5)

ax[0].set_ylim(-.1,1.1)
ax[0].legend()
ax[1].set_ylim(-.1,1.1)
ax[1].legend()

In [ ]:
for i in [25,26,28,29,32]:
    crv = crvd[ceph_names[i]]['g']
    print("{:2}: {:.3f}, {:.4f}".format(i,plist[i],max(crv[:,1])-min(crv[:,1])))

In [ ]:
for j in [0,15,31,34]:
    crv = crvd[ceph_names[j]]['g']
    print("{:2}: {:.3f}, {:.4f}".format(j,plist[j],max(crv[:,1])-min(crv[:,1])))

In [ ]:
ceph_names[34]

In [ ]:
crv = crvd[ceph_names[15]]['g']
plt.scatter(crv[:,0],crv[:,1])

In [ ]:
mcrv = np.vstack((tt,yy)).T
crv1 = mcrv[mcrv[:,0].argsort()]
#bspline fit
t,c,k = interpolate.splrep(crv1[:,0],crv1[:,1],s=5,k=5)
spline = interpolate.BSpline(t,c,k)
plt.scatter(crv1[:,0],crv1[:,1],s=3)
plt.plot(crv1[:,0],spline(crv1[:,0]),c='k')

scipy.interpolate.LSQUnivariateSpline

#10 has good coverage but too wide
#12 and #13 #14 #15 #5 #17

short:
#3, #16

bad period?
#18 #6

In [ ]:
# figall, axall = plt.subplots()
figg, axg = plt.subplots()
# figr, axr = plt.subplots()
# figi, axi = plt.subplots()
# figz, axz = plt.subplots()
count = 0
phshft = np.zeros(20)
for i in [4,9,10,11,12,16,19,27,33]:
    p = pdict[ceph_names2[i]]
    print((i,p))
    for fltr, crv in crvd[ceph_names2[i]].items():
        m = int(len(crv)*0)
        n = int(len(crv)*1)
#         axall.scatter(crv[m:n,0]/p,crv[m:n,1]/max(crv[:,1]))
        if fltr == 'g':
            count = count + 1
            axg.scatter(crv[m:n,0]/p+phshft[i],crv[m:n,1]/max(crv[:,1]),label=i,s=5)
axg.legend()
#         if fltr == 'r':
#             axr.scatter(crv[m:n,0]/p,crv[m:n,1]/max(crv[:,1]))
#         if fltr == 'i':
#             axi.scatter(crv[m:n,0]/p,crv[m:n,1]/max(crv[:,1]))
#         if fltr == 'z':
#             axz.scatter(crv[m:n,0]/p,crv[m:n,1]/max(crv[:,1]))
#             coeffs = np.polyfit(crv[m:n,0],crv[m:n,1],2)
#             y = np.array([np.sum(np.array([coeffs[i]*(j**i) for i in range(len(coeffs))])) for j in crv[m:n,0]])
#             #axz.plot(y,crv[m:n,0])

In [ ]:
mcrv = np.vstack((t2,y2)).T
crv1 = mcrv[mcrv[:,0].argsort()]
#bspline fit
t,c,k = interpolate.splrep(crv1[:,0],crv1[:,1],s=1.1,k=5)
spline = interpolate.BSpline(t,c,k)
plt.scatter(crv1[:,0],crv1[:,1],s=3)
plt.plot(crv1[:,0],spline(crv1[:,0]),c='k')

In [ ]:
figall, axall = plt.subplots()
figu, axu = plt.subplots()
figg, axg = plt.subplots()
figr, axr = plt.subplots()
figi, axi = plt.subplots()
figz, axz = plt.subplots()
for i in range(len(ceph_names)):
    p = pdict[ceph_names[i]]
    fig1, ax1 = plt.subplots()
    for fltr, crv in crvd[ceph_names[i]].items():
        axall.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]))
        ax1.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]),c=cf[fltr],label=fltr)
        if fltr == 'u':
            axu.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]))
        if fltr == 'g':
            axg.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]))
        if fltr == 'r':
            axr.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]))
        if fltr == 'i':
            axi.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]))
        if fltr == 'z':
            axz.scatter(crv[:,0]/p,crv[:,1]/max(crv[:,1]))
    ax1.set_title('{} - Phased Normalized'.format(ceph_names[i]))
    ax1.legend()
    fig1.savefig('overplot/{}_phased_norm.pdf'.format(ceph_names[i]))
    plt.close(fig1)
axu.set_title('u - Phased Normalized')
figu.savefig('overplot/u_phased_norm.pdf')
plt.close(figu)
axg.set_title('g - Phased Normalized')
figg.savefig('overplot/g_phased_norm.pdf')
plt.close(figg)
axr.set_title('r - Phased Normalized')
figr.savefig('overplot/r_phased_norm.pdf')
plt.close(figr)
axi.set_title('i - Phased Normalized')
figi.savefig('overplot/i_phased_norm.pdf')
plt.close(figi)
axz.set_title('z - Phased Normalized')
figz.savefig('overplot/z_phased_norm.pdf')
plt.close(figz)
axall.set_title('All - Phased Normalized')
figall.savefig('overplot/all_phased_norm.pdf')
plt.close(figall)